# 2. Data Exploration: Maximizing Conversion Ratio

    Author: Angel He
    Date: October 18, 2021
    
This notebook is a continuation of previous work by Angel He to explore the user and item conversion ratios. User conversion ratio is defined as the number of "queries" made by a user before the user places an order, represented as `user_conversion_ratio = num_orders / num_queries`. Item conversion ratio is defined as the number of global "queries" made on an item before the item is ordered by any user, represented as `item_conversion_ratio = num_orders / num_queries`. A query is defined as an attempt by a user to reserve an item for a fixed date range. 

A query is successful when `reservation.is_calendared = True`. This only happens when there are no other completed reservations on that item and the renter converts the queries into an order. A query is failed in every other case. Primarily, two cases arise:
1. The query conflicts with an existing successful query from another user on the platform.*
2. The query does not conflict with any existing successful queries; however, the renter doesn't convert.

## Research question: "how can we maximize each user's conversion ratio?"

Hubbub wants to maximize the conversion ratio of its users and items. A maximized conversion ratio means that number of converted orders is approximately equal to the number of queries, `num_orders ~= num_queries`. This notebook explores the difference between these previously described types of "failed queries". 

Some Questions:
* How often is an item available when each user queries for it.

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
pd.option_context("display.max_columns", None)

In [31]:
res=pd.read_csv('/content/drive/MyDrive/Hubbub/reservations.csv')
orders=pd.read_csv('/content/drive/MyDrive/Hubbub/orders.csv')
res['date_started2']=pd.to_datetime(res['date_started']) #.dt.date
res['date_ended2']=pd.to_datetime(res['date_ended']) #
res['dt_created2']=pd.to_datetime(res['dt_created']) #.dt.date
orders['res_date_start2']=pd.to_datetime(orders['res_date_start']) #.dt.date
orders['res_date_end2']=pd.to_datetime(orders['res_date_end'])
orders['date_placed2']=pd.to_datetime(orders['date_placed'])

In [32]:
res.head()

,date_started,date_ended,is_calendared,is_extended,is_in_cart,renter_id,item_id,charge,deposit,tax,dt_created,date_started2,date_ended2,dt_created2
0,2021-10-04,2021-12-19,True,False,False,2756,245,44.621315,11.155329,3.960142,2021-10-02 02:47:39.199506,2021-10-04,2021-12-19,2021-10-02 02:47:39.199506
1,2021-10-02,2021-10-03,True,False,False,2756,325,40.192813,10.048203,3.567112,2021-10-02 02:26:05.224475,2021-10-02,2021-10-03,2021-10-02 02:26:05.224475
2,2021-10-02,2021-12-18,False,False,False,2756,325,40.060297,10.015074,3.555351,2021-09-30 18:09:18.070401,2021-10-02,2021-12-18,2021-09-30 18:09:18.070401
3,2021-09-05,2021-10-02,True,False,False,2756,81,6.275663,1.568916,0.556965,2021-09-30 18:08:48.910432,2021-09-05,2021-10-02,2021-09-30 18:08:48.910432
4,2021-10-02,2021-12-19,False,False,False,2756,325,40.192813,10.048203,3.567112,2021-09-30 18:07:00.441643,2021-10-02,2021-12-19,2021-09-30 18:07:00.441643


In [34]:
# res.dtypes

In [24]:
orders.head()

,id,date_placed,is_online_pay,is_dropoff_sched,is_pickup_sched,lister_id,item_id,renter_id,res_date_start,res_date_end,res_date_start2,res_date_end2,date_placed2
0,8025,2021-10-06,False,True,False,32,375,2795,2021-10-08,2022-05-21,2021-10-08,2022-05-21,2021-10-06
1,8018,2021-09-26,False,True,False,32,206,2793,2021-09-28,2022-05-20,2021-09-28,2022-05-20,2021-09-26
2,8014,2021-09-16,False,True,False,32,147,2790,2021-09-19,2021-10-18,2021-09-19,2021-10-18,2021-09-16
3,8013,2021-09-16,False,True,False,32,205,2790,2021-09-19,2021-12-19,2021-09-19,2021-12-19,2021-09-16
4,8009,2021-09-14,False,True,False,32,335,2788,2021-09-16,2022-05-19,2021-09-16,2022-05-19,2021-09-14


In [12]:
# what percentage of unordered queries return valid? if high, there may be other factors why people aren't ordering
# make a column in res for valid
# if range of res_date_start & res_date_end from orders overlaps with the range of dates queried in reservations, then that query is invalid
# https://stackoverflow.com/questions/9044084/efficient-date-range-overlap-calculation-in-python 
# only need to check the ones where is_calendared is false

True

In [35]:
res_notcal=res[res.is_calendared==False]
res_notcal.head()

,date_started,date_ended,is_calendared,is_extended,is_in_cart,renter_id,item_id,charge,deposit,tax,dt_created,date_started2,date_ended2,dt_created2
2,2021-10-02,2021-12-18,False,False,False,2756,325,40.060297,10.015074,3.555351,2021-09-30 18:09:18.070401,2021-10-02,2021-12-18,2021-09-30 18:09:18.070401
4,2021-10-02,2021-12-19,False,False,False,2756,325,40.192813,10.048203,3.567112,2021-09-30 18:07:00.441643,2021-10-02,2021-12-19,2021-09-30 18:07:00.441643
5,2021-10-03,2021-12-12,False,False,False,2756,245,43.622891,10.905723,3.871532,2021-09-30 18:02:35.575701,2021-10-03,2021-12-12,2021-09-30 18:02:35.575701
6,2021-10-03,2021-12-12,False,False,False,2756,325,39.031008,9.757752,3.464002,2021-09-30 12:46:57.768529,2021-10-03,2021-12-12,2021-09-30 12:46:57.768529
9,2021-10-01,2021-12-22,False,False,True,2784,332,21.541835,5.385459,1.911838,2021-09-29 16:28:57.900269,2021-10-01,2021-12-22,2021-09-29 16:28:57.900269


In [94]:
valid_li=[]
for i in res_notcal.index: #[:10]: 
  valid=[]
  item=res_notcal.loc[i,'item_id']
  querying_renter=res_notcal.loc[i,'renter_id']
  res_start=res_notcal.loc[i,'date_started2']
  res_end=res_notcal.loc[i,'date_ended2']
  query_date=res_notcal.loc[i,'dt_created2']
  orders_item=orders[(orders.item_id==item) & (orders.date_placed2<=query_date) & (orders.renter_id!=querying_renter)] #### since we don't have exact time for order placement, the query user needs to be different than the order user? for eg. user queries for 10/1 - 10/11 on 9/20 9:30am, but user orders for 10/1-10/11 on 9/20 9pm, if we don't exclude that order from the entries, this query will be invalid even though it was actually valid ####
  orders_start=orders_item.res_date_start2
  orders_end=orders_item.res_date_end2
  # print('res df: \n',res_notcal.loc[i,:])
  # print('res start & end: \n',res_start, res_end)
  # print('orders df: \n',orders_item)
  # print('orders start: \n',orders_start)
  # print('orders end: \n', orders_end) #, '\n')
  orders_num=len(orders_item)
  for j in orders_item.index: 
    latest_start=max(res_start,orders_start[j])
    earliest_end=min(res_end,orders_end[j])
    # print('latest start: ',latest_start)
    # print('earliest end: ', earliest_end)
    # print('earliest end - latest start: ',earliest_end-latest_start,type(earliest_end-latest_start))
    if earliest_end-latest_start >= datetime.timedelta(days=0): #### query date range can start at most as early as the next day after previous order ends, is this a correct assumption? ####
      # print('date ranges overlap, query returns invalid \n') 
      # append invalid to validity column in res
      valid.append(0)
      break
  if len(valid)==0: # append valid to validity column in res
    valid.append(1)
  valid_li.append(valid[0])
  # print(valid,'\n')
   

In [95]:
res_notcal['valid']=valid_li
res_notcal

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date_started,date_ended,is_calendared,is_extended,is_in_cart,renter_id,item_id,charge,deposit,tax,dt_created,date_started2,date_ended2,dt_created2,valid
2,2021-10-02,2021-12-18,False,False,False,2756,325,40.060297,10.015074,3.555351,2021-09-30 18:09:18.070401,2021-10-02,2021-12-18,2021-09-30 18:09:18.070401,1
4,2021-10-02,2021-12-19,False,False,False,2756,325,40.192813,10.048203,3.567112,2021-09-30 18:07:00.441643,2021-10-02,2021-12-19,2021-09-30 18:07:00.441643,1
5,2021-10-03,2021-12-12,False,False,False,2756,245,43.622891,10.905723,3.871532,2021-09-30 18:02:35.575701,2021-10-03,2021-12-12,2021-09-30 18:02:35.575701,1
6,2021-10-03,2021-12-12,False,False,False,2756,325,39.031008,9.757752,3.464002,2021-09-30 12:46:57.768529,2021-10-03,2021-12-12,2021-09-30 12:46:57.768529,1
9,2021-10-01,2021-12-22,False,False,True,2784,332,21.541835,5.385459,1.911838,2021-09-29 16:28:57.900269,2021-10-01,2021-12-22,2021-09-29 16:28:57.900269,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2021-05-14,2021-08-15,False,False,False,281,213,54.005248,13.501312,0.000000,2021-07-19 11:01:21.375775,2021-05-14,2021-08-15,2021-07-19 11:01:21.375775,1
484,2021-05-12,2021-08-12,False,False,False,280,249,46.640896,11.660224,0.000000,2021-07-19 11:01:18.133208,2021-05-12,2021-08-12,2021-07-19 11:01:18.133208,1
486,2021-06-15,2021-07-31,False,False,False,161,230,19.824119,4.956030,0.000000,2021-07-19 11:01:17.17263,2021-06-15,2021-07-31,2021-07-19 11:01:17.172630,1
493,2021-06-27,2021-06-28,False,False,False,324,236,9.000000,2.250000,0.000000,2021-07-19 11:01:13.847357,2021-06-27,2021-06-28,2021-07-19 11:01:13.847357,1


In [96]:
np.sum(res_notcal.valid)/len(res_notcal) # pct times query valid in res_notcal

0.5243553008595988

In [97]:
1-np.sum(res_notcal.valid)/len(res_notcal) # pct times query invalid in res_notcal

0.4756446991404012

In [98]:
# check
orders[orders.item_id==325] # case when renter id is in res and also in orders

,id,date_placed,is_online_pay,is_dropoff_sched,is_pickup_sched,lister_id,item_id,renter_id,res_date_start,res_date_end,res_date_start2,res_date_end2,date_placed2
43,8023,2021-09-30,False,True,False,32,325,2756,2021-10-02,2021-10-03,2021-10-02,2021-10-03,2021-09-30
185,7992,2021-09-06,False,True,True,32,325,56,2021-09-08,2021-10-01,2021-09-08,2021-10-01,2021-09-06


In [102]:
orders[orders.item_id==258]

,id,date_placed,is_online_pay,is_dropoff_sched,is_pickup_sched,lister_id,item_id,renter_id,res_date_start,res_date_end,res_date_start2,res_date_end2,date_placed2
56,245,2021-06-28,False,True,False,32,258,325,2021-07-07,2021-07-14,2021-07-07,2021-07-14,2021-06-28
204,7968,2021-09-01,False,True,False,32,258,29,2021-09-03,2021-09-30,2021-09-03,2021-09-30,2021-09-01


In [100]:
with pd.option_context("display.max_rows", None):
  display(res_notcal)

,date_started,date_ended,is_calendared,is_extended,is_in_cart,renter_id,item_id,charge,deposit,tax,dt_created,date_started2,date_ended2,dt_created2,valid
2,2021-10-02,2021-12-18,False,False,False,2756,325,40.060297,10.015074,3.555351,2021-09-30 18:09:18.070401,2021-10-02,2021-12-18,2021-09-30 18:09:18.070401,1
4,2021-10-02,2021-12-19,False,False,False,2756,325,40.192813,10.048203,3.567112,2021-09-30 18:07:00.441643,2021-10-02,2021-12-19,2021-09-30 18:07:00.441643,1
5,2021-10-03,2021-12-12,False,False,False,2756,245,43.622891,10.905723,3.871532,2021-09-30 18:02:35.575701,2021-10-03,2021-12-12,2021-09-30 18:02:35.575701,1
6,2021-10-03,2021-12-12,False,False,False,2756,325,39.031008,9.757752,3.464002,2021-09-30 12:46:57.768529,2021-10-03,2021-12-12,2021-09-30 12:46:57.768529,1
9,2021-10-01,2021-12-22,False,False,True,2784,332,21.541835,5.385459,1.911838,2021-09-29 16:28:57.900269,2021-10-01,2021-12-22,2021-09-29 16:28:57.900269,1
10,2021-09-30,2022-05-31,False,False,False,326,129,18.797735,4.699434,1.668299,2021-09-28 19:11:05.261076,2021-09-30,2022-05-31,2021-09-28 19:11:05.261076,1
15,2021-09-29,2021-10-06,False,False,False,2794,29,2.426790,0.606698,0.215378,2021-09-27 14:28:12.315599,2021-09-29,2021-10-06,2021-09-27 14:28:12.315599,1
16,2021-09-29,2021-10-12,False,False,False,2794,15,12.885103,3.221276,1.143553,2021-09-27 04:34:49.820477,2021-09-29,2021-10-12,2021-09-27 04:34:49.820477,1
19,2021-09-24,2021-12-12,False,False,False,2756,167,30.834449,7.708612,2.736557,2021-09-21 18:30:33.098122,2021-09-24,2021-12-12,2021-09-21 18:30:33.098122,1
20,2021-09-22,2021-12-17,False,False,True,2787,270,30.007211,7.501803,2.663140,2021-09-20 19:01:22.360039,2021-09-22,2021-12-17,2021-09-20 19:01:22.360039,1
